In [1]:
import rail
from rail.estimation.algos.deepdisc import *
from rail.core.data import TableHandle, JsonHandle, ModelHandle
from rail.core.stage import RailStage
import torch.multiprocessing as mp

In [2]:
mp.set_start_method('spawn')


In [3]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [4]:
model_file = "./test_informer.pkl"
model_handle = DS.add_data("model", data=None, handle_class=ModelHandle, path=model_file)
test_file_for_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_test_small.hdf5"
test_handle_for_chunks = DS.add_data("testing", data=None, handle_class=TableHandle, path=test_file_for_chunks)
metadatafile_with_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/test_metadata_example.hdf5"
metadatahandle_with_chunks = DS.add_data("metadata", data=None, handle_class=Hdf5Handle, path=metadatafile_with_chunks)

cfgfile = "./configs/solo/solo_swin_DC2.py"

In [5]:
deep_estimation_dict = dict(
    chunk_size=5,
    output_dir="./",
    cfgfile = cfgfile,
    zmin=0,
    zmax=5,
    nzbins=200,
    output_mode='default',
)

EstimatorWithChunks = DeepDiscPDFEstimatorWithChunking.make_stage(
    name="DeepDiscEstimatorWithChunks",
    model=model_file,
    **deep_estimation_dict,
)

In [6]:
results_from_chunks = EstimatorWithChunks.estimate(test_handle_for_chunks, metadatahandle_with_chunks)

/home/g4merz/.conda/envs/ddrailnv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Caching data
Processing chunk (start:end) - (0:5)
Matching objects - rank: 0
[01/15 15:21:26 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
Matching objects - rank: 1
[01/15 15:21:26 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[01/15 15:21:26 d2.data.common]: Serializing 5 elements to byte tensors and concatenating them all ...
[01/15 15:21:26 d2.data.common]: Serializing 5 elements to byte tensors and concatenating them all ...
[01/15 15:21:26 d2.data.common]: Serialized dataset takes 1.29 MiB
[01/15 15:21:26 d2.data.common]: Serialized dataset takes 1.29 MiB


/home/g4merz/.conda/envs/ddrailnv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/g4merz/.conda/envs/ddrailnv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Returned pdfs - rank: 1
pre-cast pdfs type: <class 'numpy.ndarray'>, pdfs value: [0.         0.02512563 0.05025126 0.07537688 0.10050251 0.12562814
 0.15075377 0.1758794  0.20100503 0.22613065 0.25125628 0.27638191
 0.30150754 0.32663317 0.35175879 0.37688442 0.40201005 0.42713568
 0.45226131 0.47738693 0.50251256 0.52763819 0.55276382 0.57788945
 0.60301508 0.6281407  0.65326633 0.67839196 0.70351759 0.72864322
 0.75376884 0.77889447 0.8040201  0.82914573 0.85427136 0.87939698
 0.90452261 0.92964824 0.95477387 0.9798995  1.00502513 1.03015075
 1.05527638 1.08040201 1.10552764 1.13065327 1.15577889 1.18090452
 1.20603015 1.23115578 1.25628141 1.28140704 1.30653266 1.33165829
 1.35678392 1.38190955 1.40703518 1.4321608  1.45728643 1.48241206
 1.50753769 1.53266332 1.55778894 1.58291457 1.6080402  1.63316583
 1.65829146 1.68341709 1.70854271 1.73366834 1.75879397 1.7839196
 1.80904523 1.83417085 1.85929648 1.88442211 1.90954774 1.93467337
 1.95979899 1.98492462 2.01005025 2.03517588 2.06

/home/g4merz/.conda/envs/ddrailnv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/g4merz/.conda/envs/ddrailnv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Returned pdfs - rank: 1
pre-cast pdfs type: <class 'numpy.ndarray'>, pdfs value: [0.         0.02512563 0.05025126 0.07537688 0.10050251 0.12562814
 0.15075377 0.1758794  0.20100503 0.22613065 0.25125628 0.27638191
 0.30150754 0.32663317 0.35175879 0.37688442 0.40201005 0.42713568
 0.45226131 0.47738693 0.50251256 0.52763819 0.55276382 0.57788945
 0.60301508 0.6281407  0.65326633 0.67839196 0.70351759 0.72864322
 0.75376884 0.77889447 0.8040201  0.82914573 0.85427136 0.87939698
 0.90452261 0.92964824 0.95477387 0.9798995  1.00502513 1.03015075
 1.05527638 1.08040201 1.10552764 1.13065327 1.15577889 1.18090452
 1.20603015 1.23115578 1.25628141 1.28140704 1.30653266 1.33165829
 1.35678392 1.38190955 1.40703518 1.4321608  1.45728643 1.48241206
 1.50753769 1.53266332 1.55778894 1.58291457 1.6080402  1.63316583
 1.65829146 1.68341709 1.70854271 1.73366834 1.75879397 1.7839196
 1.80904523 1.83417085 1.85929648 1.88442211 1.90954774 1.93467337
 1.95979899 1.98492462 2.01005025 2.03517588 2.06

In [7]:
res_ens = results_from_chunks.read()
res_ens.npdf

2

In [8]:
res.an

NameError: name 'res' is not defined

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
x = np.linspace(0,3,100)
ax.plot(x, res_ens[0].pdf(x)[0])
ax.plot(x, res_ens[1].pdf(x)[0])